In [ ]:
import numpy as np
import pandas as pd
import panel as pn
import seaborn as sns
import time

from bokeh.models import Range1d, ColumnDataSource, HoverTool, LinearColorMapper, ColorBar, WheelZoomTool, CDSView, IndexFilter, BasicTicker, FixedTicker, PrintfTickFormatter, FuncTickFormatter
from bokeh.plotting import figure, show
from bokeh.transform import linear_cmap
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

from umap import UMAP
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KDTree, BallTree
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
import sklearn.datasets

from scipy import linalg

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, MaxPooling2D, Conv2D, Flatten, Dropout
from tensorflow.keras.initializers import VarianceScaling, Constant
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist
from tensorflow.keras.metrics import CategoricalCrossentropy
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
print(tf.config.list_physical_devices())
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.disable_eager_execution()
tf.random.set_seed(42)

In [ ]:
def rgb_to_hex(c):
    h = '#'
    for i in c:
        h = h + '{0:02X}'.format(int(i*255))
    return h
    
class Params():
    total_width = 1400
    total_height = 500
    bot_height = 1000
    header_height = 32
    emb_width = 500
    fea_width = 160
    tbl_width = 100
    hor_width = total_width-(emb_width+fea_width+tbl_width)
    red_res = 500
    pdp_res = 50
    dot_size = 6
    
    dataset_name = 'breast'
    neighbors = 100
    classifier = 'TensorFlow100'
    
    palette = []
    cm = None
    num_colors = 0
    sat_palette = []
    sat_cm = None
    
    def __init__(self):
#         sat_color_list = ['#d95f02','#1b9e77','#7570b3','#e6ab02','#66a61e','#e7298a','#a6761d','#666666'] #1
#        sat_color_list = ['#ff6023','#36c296','#5a7dcc','#e7298a','#66a61e','#e6ab02','#a6761d','#666666'] #1.1
#         sat_color_list = ['#5da5da','#faa43a','#60bd68','#f17cb0','#b2912f','#b276b2','#decf3f','#f15854'] #2
        sat_color_list = ['#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd','#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf'] # Cat10
#         color_list = ['#fc8d62','#66c2a5','#8da0cb','#ffd92f','#a6d854','#e78ac3','#e5c494','#b3b3b3'] #1
        color_list = [rgb_to_hex(sns.light_palette(c, n_colors=6)[3]) for c in sat_color_list]
        self.num_colors = len(color_list)
        for i in range(self.num_colors):
            self.palette += [rgb_to_hex(c) for c in sns.light_palette(color_list[i], n_colors=6)][1:]
        self.sat_palette = sat_color_list
        self.cm = ListedColormap(colors=self.palette)
        self.sat_cm = ListedColormap(colors=self.sat_palette)
            
params = Params()

class Dataset():
    name = ''
    data = None 
    datasource = ColumnDataSource()
    view_filter = IndexFilter()
    wrong_filter = IndexFilter()
    view = CDSView()
    labels = None
    features = []
    selected_features = []
    classes = None
    categories = None
    trees = []
    
    bounds = []
    point = []
    
    def load_dataset(self, name):
        if name=="iris":
            dataset = sklearn.datasets.load_iris()
            data = dataset.data
            target = dataset.target
            features = dataset.feature_names
            classes = dataset.target_names
        if name=="diabetes":
            X = pd.read_csv("./data/diabetes.csv")
            data = X.to_numpy()[:,:-1]
            target = np.invert(np.array(X.to_numpy()[:,-1], dtype=bool))
            features = X.columns[:-1]
            classes = ["diabetes","no diabetes"]
        if name=="shuttle":
            X = np.loadtxt("./data/shuttle/shuttle-train.txt")
            data = X[:,:-1]
            target = X[:,-1].astype(int)-1
            features = ["1","2","3","4","5","6","7","8","9"]
            classes = ["Rad Flow", "Fpv Close", "Fpv Open", "High", "Bypass", "Bpv Close", "Bpv Open"]
        if name=="robot24":
            X = pd.read_csv("./data/robot/sensor_readings_24.txt")
            data = X.to_numpy()[:,:-1]
            target = X.to_numpy()[:,-1].astype(int)-1
            features = ["180°","-165°","-150°","-135°","-120°","-105°","-90°","-75°","-60°","-45°","-30°","-15°","0°",
                        "15°","30°","45°","60°","75°","90°","105°","120°","135°","150°","165°"]
            classes = ["Move-Forward", "Slight-Right-Turn", "Sharp-Right-Turn", "Slight-Left-Turn"]
        if name=="mnist":
            (x_train, y_train), (x_test, y_test) = mnist.load_data(path="mnist.npz")
            data = x_train.reshape(x_train.shape[0], -1)
            target = y_train
            features = [str(i) for i in range(data.shape[1])]
            classes = [str(i) for i in range(10)]
        if name=="breast":
            dataset = sklearn.datasets.load_breast_cancer()
            data = dataset.data[:,:18]
            target = dataset.target
            features = dataset.feature_names[:18]
            classes = dataset.target_names
            
        if not isinstance(features, list):
            features = features.tolist()
            
        if name=="shuttle" or name=="robot24" or name=="robot4":
            n = 400
            new_data = np.zeros((0,data.shape[1]))
            new_target = np.zeros(0, dtype=int)
            np.random.seed(0)
            for i in np.unique(target):
                possible_inds = np.where(target == i)[0]
                rnd_ind = np.random.choice(possible_inds, min(n,possible_inds.shape[0]), replace=False)
                new_data = np.append(new_data, data[rnd_ind,:], axis=0)
                new_target = np.append(new_target, target[rnd_ind])
            data = new_data
            target = new_target
            
        return pd.DataFrame(data, columns=features), target, features, classes
    
    def update(self, name):
        self.name = name
        self.data,self.labels,self.features,self.classes = self.load_dataset(self.name)
        self.selected_features = self.features
        
        self.data['color']      = ['#444444']*len(self.data.index)
        self.data['sat_color']  = ['#444444']*len(self.data.index)
        self.data['line_color'] = ['#444444']*len(self.data.index)
        self.data['size'] = [params.dot_size]*len(self.data.index)
        self.data['x']  = [0]*len(self.data.index)
        self.data['y']  = [0]*len(self.data.index)
        self.data['x1'] = [0]*len(self.data.index)
        self.data['y1'] = [0]*len(self.data.index)
        self.datasource.selected.indices = [0] 
        self.datasource.selected.indices = []
        self.datasource.data = self.data
        self.view_filter.indices  = list(self.data.index)
        self.wrong_filter.indices = list(self.data.index)
        self.view = CDSView(source=self.datasource, filters=[self.view_filter])
        
#         self.bounds = [dataset.data[dataset.features].quantile(0.05).to_numpy(), dataset.data[dataset.features].quantile(0.95).to_numpy()]
        self.bounds = [dataset.data[dataset.features].min().to_numpy(), dataset.data[dataset.features].max().to_numpy()]
        self.point = dataset.data[dataset.features].mean()
        print("--- data loaded", self.data.shape)

dataset = Dataset()
dataset.update(params.dataset_name)

class TFModel():
    typ = None
    
    def __init__(self, typ, input_shape):
        # define keras model
        self.model = Sequential()
        if typ == '2048':
            self.model.add(Dense(2048, input_dim=input_shape, activation='relu'))
            self.model.add(Dense(2048, activation='relu'))
            self.model.add(Dense(2048, activation='relu'))
            self.model.add(Dense(len(dataset.classes), activation='softmax'))
            self.model.compile(loss='mse', optimizer='sgd', metrics=["accuracy"])
        elif typ == 'Conv':
            self.typ = 'Conv'
            self.model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
            self.model.add(MaxPooling2D((2, 2)))
            self.model.add(Conv2D(64, (3, 3), activation='relu'))
            self.model.add(MaxPooling2D((2, 2)))
            self.model.add(Conv2D(64, (3, 3), activation='relu'))
            self.model.add(Flatten())
            self.model.add(Dropout(0.5))
            self.model.add(Dense(len(dataset.classes), activation="softmax"))
            self.model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
        #self.model.compile(optimizer='adam', loss=SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
        elif typ == '100':
            self.model.add(Dense(100, input_dim=input_shape, activation='relu'))
            self.model.add(Dense(100, activation='relu'))
            self.model.add(Dense(100, activation='relu'))
            self.model.add(Dense(len(dataset.classes), activation='softmax'))
            self.model.compile(loss='mse', optimizer='adam', metrics=["accuracy"])
            
    def fit(self, X, Y):
        # encode class values as integers
        encoder = LabelEncoder()
        encoder.fit(Y)
        encoded_Y = encoder.transform(Y)
        # convert integers to dummy variables (i.e. one hot encoded)
        dummy_y = to_categorical(encoded_Y)
        if self.typ == 'Conv':
            # transform to image proportions
            dummy_x = X.reshape(-1,28,28,1)
        else:
            dummy_x = X
        # train model
        tic = time.perf_counter()
        self.model.fit(dummy_x, dummy_y, epochs=200, batch_size=200, verbose=0)
        toc = time.perf_counter()
        print("Train Model:",toc-tic,"s")
        print('Model accuracy:', self.model.evaluate(dummy_x, dummy_y))
    
    def predict_proba(self, X):
        if self.typ == 'Conv':
            # transform to image proportions
            dummy_x = X.reshape(-1,28,28,1)
        else:
            dummy_x = X
        return self.model.predict(dummy_x)
    
    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=1)

class Predictor():
    pip = None
    
    def update_pip(self, classifier_type, input_shape=None):
        if classifier_type == "RandomForest":
            self.pip = Pipeline([('scaler', StandardScaler()), 
                                 ('classificator', RandomForestClassifier(
                                      random_state=1, min_samples_split = 4, min_samples_leaf = 3))])
        elif classifier_type == "NeuralNetwork":
            self.pip = Pipeline([('scaler', StandardScaler()), 
                                 ('classificator', MLPClassifier(hidden_layer_sizes=(100,100,100), 
                                                                 max_iter=5000, random_state=1))])
        elif classifier_type == "NeuralNetworkBig":
            self.pip = Pipeline([('scaler', StandardScaler()), 
                                 ('classificator', MLPClassifier(hidden_layer_sizes=(2048,2048,2048), 
                                                                 max_iter=5000, random_state=1))])
        elif classifier_type == "TensorFlow100":
            self.pip = Pipeline([('scaler', StandardScaler()), ('classificator', TFModel('100',input_shape))])
        elif classifier_type == "TensorFlowConv":
            self.pip = Pipeline([('scaler', StandardScaler()), ('classificator', TFModel('Conv',input_shape))])
        elif classifier_type == "TensorFlow2048":
            self.pip = Pipeline([('scaler', StandardScaler()), ('classificator', TFModel('2048',input_shape))])
        else: print("unkown classifier type:", classifier_type)
        
    def update_data(self, dataset, params):
        # train predictor on dataset
        tic = time.perf_counter()
        self.pip.fit(dataset.data[dataset.features], dataset.labels)
        toc = time.perf_counter()
        print("--- predictor trained:", toc-tic,"s")
        # save predicitons in dataset
        dataset.data['prob'] = self.pip.predict_proba(dataset.data[dataset.features]).tolist()
        dataset.data['maxprob'] = np.max(np.array(dataset.data['prob'].tolist()), axis=1)
        dataset.datasource.data['prob'] = dataset.data['prob']
        dataset.datasource.data['maxprob'] = dataset.data['maxprob']
        # compute colors of dataset
        dataset.data['most_prob_class'] = np.argmax(np.array(dataset.data['prob'].tolist()), axis=1)
        colorvalues = (np.clip(dataset.data['maxprob'].to_numpy(),0.51,0.99) - 0.5) * 2 + dataset.data['most_prob_class'].to_numpy()
        dataset.data['color'] = [rgb_to_hex(params.cm(i/params.num_colors)) for i in colorvalues]
        dataset.data['sat_color'] = [params.sat_palette[i] for i in dataset.data['most_prob_class'].to_numpy()]
        dataset.data['line_color'] = dataset.data['sat_color']
        dataset.datasource.data['color'] = dataset.data['color']
        dataset.datasource.data['sat_color'] = dataset.data['sat_color']
        dataset.datasource.data['line_color'] = dataset.data['sat_color']
        
        # compute wrongly predicted indices
        dataset.wrong_filter.indices = np.where(np.not_equal(dataset.data['most_prob_class'].to_numpy(), dataset.labels))[0].tolist()
#         dataset.datasource.data['target_color'] = np.where(
#             dataset.data['most_prob_class'].to_numpy() != dataset.labels,
#             [rgb_to_hex(params.sat_cm(i/params.num_colors)) for i in (dataset.labels+0.99)], 
#             np.full(len(dataset.data),'#333333'))
        dataset.datasource.data['target_color'] = [params.sat_palette[i] for i in dataset.labels]
    
        # create nearest neighbor search structure per class
        dataset.trees = []
        for i in range(len(dataset.classes)):
            index_map = np.where(dataset.data['most_prob_class'].to_numpy() == i)[0]
            if len(index_map) == 0: 
                dataset.trees.append(([], None))
                continue
            class_data = dataset.data.loc[index_map,dataset.features]
            scaled_class_data = self.pip['scaler'].transform(class_data)
            dataset.trees.append( (index_map, BallTree(scaled_class_data)) )
        
#import tensorflow as tf
#if tf.test.gpu_device_name(): 
#    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
#else:
#    print("Please install GPU version of TF")

predictor = Predictor()
predictor.update_pip(params.classifier, input_shape=len(dataset.features))
predictor.update_data(dataset, params)

In [ ]:
from embedding_util2_1 import Shifter, SVD

class Embedding():
    
    def __init__(self, dataset, nn_inds=[], point=None):
        self.emb = Pipeline([('shifter', Shifter()), ('scaler', StandardScaler()), ('pca', SVD())])
        
        if len(nn_inds) < 1:
            nn_inds = dataset.data.index.tolist()
               
        train_set = dataset.data.loc[nn_inds,dataset.selected_features]
        self.emb.fit(train_set)
            
        shift_bounds = [[dataset.bounds[j][i] for i,f in enumerate(dataset.features) if (f in dataset.selected_features)] for j in [0,1]]
        self.emb['shifter'].set_bounds(shift_bounds) 
        if point is not None:
            # shift plane to focus point
            shift = point[dataset.selected_features].values.astype('float64') - self.emb.inverse_transform(self.emb.transform([point[dataset.selected_features]]))[0]
            self.emb['shifter'].set_by(shift)
    
    def fit(self, X, y=None):
        self.emb.fit(X, y)
            
    def transform(self, X):
        return self.emb.transform(X)
    
    def inverse_transform(self, X):
        return self.emb.inverse_transform()
    
    
def update_non_linear_view(p, predictor, method="PCA", neighbors=20, sel_feats_only=False, reverse='PCA'):
    dataset.selected_features = dataset.features
    # reduce data with non-linear DR
    if method == "UMAP":
        nl_pip = Pipeline([('scaler', StandardScaler()), ('reducer', UMAP(n_neighbors=neighbors, random_state=42))])
    elif method == "t-SNE":
        nl_pip = Pipeline([('scaler', StandardScaler()), ('reducer', TSNE(perplexity=neighbors, random_state=42))])
    elif method == "PCA":
        embedding = Embedding(dataset)
        nl_pip = Pipeline([('scaler', embedding.emb[:2]), ('reducer', embedding.emb['pca'])])
    if sel_feats_only:
        reduced = nl_pip.fit_transform(dataset.data[dataset.selected_features])
    else:
        tic = time.perf_counter()
        reduced = nl_pip.fit_transform(dataset.data[dataset.features])
        toc = time.perf_counter()
        print(method,dataset.data.shape[0]//1000,"K samples:",toc-tic,"s")
    
    dataset.data['x1'] = reduced[:,0]
    dataset.data['y1'] = reduced[:,1]
    
    # update plot bounds
    bx = max(dataset.data['x1'])-min(dataset.data['x1'])
    by = max(dataset.data['y1'])-min(dataset.data['y1'])
    x_min = min(dataset.data['x1'])-0.1*bx
    x_max = max(dataset.data['x1'])+0.1*bx
    y_min = min(dataset.data['y1'])-0.1*by
    y_max = max(dataset.data['y1'])+0.1*by
    p.x_range.update(start=x_min, end=x_max)
    p.y_range.update(start=y_min, end=y_max)
    
    # update plot datasource
    #dataset.datasource.data['x1'] = reduced[:,0]
    #dataset.datasource.data['y1'] = reduced[:,1]
    
    ### create background map ###
    
    # create samples
    N = 200
    px = np.linspace(x_min, x_max, num=N)
    py = np.linspace(y_min, y_max, num=N)
    xx, yy = np.meshgrid(px,py)
    grid = np.c_[xx.ravel(), yy.ravel()]
    
    ### iLAMP ###
    if reverse == 'iLAMP':
        # find map-neighbors
        kdtree = KDTree(reduced)

        tic = time.perf_counter()
        invp = np.zeros((N*N, len(dataset.features)))
        for i, point in enumerate(grid):
            dists, Ys_inds = kdtree.query(point.reshape(1,-1), k=20)
            # remove queried point from set
            dists, Ys_inds = dists.T, Ys_inds[0]
            Ys = reduced[Ys_inds]
            Xs = nl_pip['scaler'].transform(dataset.data.loc[Ys_inds,dataset.features].to_numpy())
            # equation (2) + eliminate division by zero
            alphas = 1 / np.maximum(np.square(dists), 0.001)
            alpha_sum = np.sum(alphas)
            # equation (3)
            x_tilde = np.sum(alphas * Xs, axis=0) / alpha_sum
            y_tilde = np.sum(alphas * Ys, axis=0) / alpha_sum
            # equation (4)
            x_hat = Xs - x_tilde
            y_hat = Ys - y_tilde
            # equation (5)
            A = np.sqrt(alphas) * y_hat
            B = np.sqrt(alphas) * x_hat
            # equation (7)
            U, D, VH = linalg.svd(A.T @ B, full_matrices=False)
            M = U @ VH
            # equation (8)
            f_p = ((point - y_tilde) @ M) + x_tilde
            invp[i] = f_p
            if (i-1) % 1000 == 0: 
                tuc = time.perf_counter()
                print("iLAMP:",i,"/",N*N,"in",tuc-tic,"s")
        invp = nl_pip['scaler'].inverse_transform(invp)
        toc = time.perf_counter()
        print("iLAMP Samples:",toc-tic,"s")
        
        
    elif reverse == 'NNInv':
        ### NNInv ###

        # prepare data transformations
        unit_scaler = MinMaxScaler()
        if sel_feats_only:
            target = unit_scaler.fit_transform(dataset.data[dataset.selected_features])
        else:
            target = unit_scaler.fit_transform(dataset.data[dataset.features])

        # define keras model
    #     var_init = VarianceScaling(scale=1.0, mode="fan_in", distribution="uniform", seed=42)
    #     const_init = Constant(0.01)
        model = Sequential()
        if False:
            model.add(Dense(2048, input_dim=2, activation='relu'))
            model.add(Dense(2048, activation='relu'))
            model.add(Dense(2048, activation='relu'))
        else:
            model.add(Dense(100, input_dim=2, activation='relu'))
            model.add(Dense(100, activation='relu'))
            model.add(Dense(100, activation='relu'))
        if sel_feats_only:
            model.add(Dense(len(dataset.selected_features), activation='sigmoid'))
        else:
            model.add(Dense(len(dataset.features), activation='sigmoid'))

        model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

        # train model
        permutation = np.random.rand(reduced.shape[0]).argsort()
        earlystop = EarlyStopping(patience=5)

        tic = time.perf_counter()
        #model.fit(reduced[permutation], target[permutation], epochs=150, batch_size=32, validation_split=0.25, callbacks=[earlystop])
        model.fit(reduced[permutation], target[permutation], epochs=150, batch_size=200, verbose=0)
        
        toc = time.perf_counter()
        print("Train Model:",toc-tic,"s")

        # save model
        #model.save('NNInv')

        # load model
        #model = load_model('NNInv')

        # infer samples
        tic = time.perf_counter()
        invp = model.predict(grid)
        invp = unit_scaler.inverse_transform(invp)
        toc = time.perf_counter()
        print("Infer",(N*N)/1000,"K samples:",toc-tic,"s")
    
    elif reverse == 'PCA' or reverse == 'PCA_filtered':
        invp = nl_pip.inverse_transform(grid)
    
    # predict samples
    tic = time.perf_counter()
    probs = predictor.predict_proba(invp)
    toc = time.perf_counter()
    print("Predict Samples:",toc-tic,"s")
    
    # update image
    img_src = (np.clip(np.max(probs, axis=1),0.501,0.999) - 0.5) * 2 + np.argmax(probs, axis=1) 
    source = p.select('image').data_source
    source.data['image'] = [img_src.reshape((N,N))]
    source.data['x'] = [x_min]
    source.data['y'] = [y_min]
    source.data['dw'] = [x_max-x_min]
    source.data['dh'] = [y_max-y_min]
    
    def bfs(img,start):
        # Breadth-first search to find pixels that are True
        if img[start[1],start[0]]:
            return start
        visited = set()
        w, h = img.shape[0]-1, img.shape[1]-1
        queue = [start]
        while queue:
            x, y = queue.pop(0)
            neighbors = []
            if x<w:
                neighbors.append((x+1,y))
            if x>0:
                neighbors.append((x-1,y))
            if y<h:
                neighbors.append((x,y+1))
            if y>0:
                neighbors.append((x,y-1))   
            for n in neighbors:
                if img[n[1],n[0]]:
                    return n
                if n not in visited:
                    visited.add(n)
                    queue.append(n)
        return None
    
    
    cf_pix_diff = np.zeros(len(dataset.data))
    cf_diff = np.zeros(len(dataset.data))
    cfs = np.zeros((len(dataset.data),len(dataset.features)))
    
    if reverse == 'PCA' or reverse == 'PCA_filtered':
        for i, point in dataset.data.iterrows():
        #for i, point in [(0,dataset.data.loc[90,:])]:
            # find balanced NN
            def find_nn(p, num_nn=100):
                # find nearest neighbors distributed: 1/2 own class, 1/2 other classes
                own_class = np.argmax(p['prob']) if p.name != None else np.argmax(predictor.pip.predict_proba([p[dataset.features]])[0])
                own_nn = np.array([])
                other_dist = np.full((len(dataset.classes), num_nn//2), np.inf)
                other_nn   = np.full((len(dataset.classes), num_nn//2), -1)
                for j in range(len(dataset.classes)):
                    if dataset.trees[j][1] is None:
                        continue
                    dist, ind = dataset.trees[j][1].query(
                        predictor['scaler'].transform([p[dataset.features]]), k=min(num_nn//2,len(dataset.trees[j][0])))
                    if j == own_class:
                        own_nn = np.append(own_nn, dataset.trees[j][0][ind])
                    else:
                        other_dist[j,:dist.shape[1]] = dist[0]
                        other_nn[j,:dist.shape[1]] = dataset.trees[j][0][ind[0]]
                other_nn = other_nn[np.unravel_index(np.argsort(other_dist, axis=None), other_dist.shape)]
                other_nn = np.delete(other_nn, np.where(other_nn == -1))
                other_nn = other_nn[:min(num_nn//2, len(other_nn))]
                nn = np.append(own_nn, other_nn).astype(int).tolist()
                return nn
            t1 = time.perf_counter()
            nn_inds = find_nn(point)
            t2 = time.perf_counter()
            
            # find important features
            if reverse == 'PCA_filtered':
                importances = permutation_importance(predictor, dataset.data.loc[nn_inds,dataset.features].to_numpy(), 
                                                     np.argmax(np.array(dataset.data.loc[nn_inds,'prob'].tolist()), axis=1),
                                                     n_repeats=5, random_state=0, scoring='accuracy').importances_mean
                if np.any((importances != 0)):
                    importances /= np.sum(importances)
                else:
                    print('ERROR: computing importances lead to zeros only')
                important_indices = np.where(importances > 0.8/len(dataset.features))[0]
                #for j in range(10):
                #    if len(important_indices) >= 2: break
                #    print('strong importance distribution', i, importances)
                #    important_indices = np.where(importances > (0.5**j)/len(dataset.features))[0]
                #    print('take indices', important_indices)
                if len(important_indices) < 2:
                    print(i, 'take indices', important_indices)
                dataset.selected_features = np.array(dataset.features)[important_indices].tolist()
            else:
                important_indices = list(range(len(dataset.features)))
                
            # create neighborhood embedding
            t3 = time.perf_counter()
            nn_emb = Embedding(dataset, nn_inds, point)
            t4 = time.perf_counter()
            nn_pip = Pipeline([('scaler', nn_emb.emb[:2]), ('reducer', nn_emb.emb['pca'])])
            
            # adjust image to local version
            reduced = np.zeros((len(dataset.data),2))
            reduced[nn_inds,:] = nn_pip.transform(dataset.data[dataset.selected_features])[nn_inds,:]
            x_min = min(reduced[:,0])
            x_max = max(reduced[:,0])
            y_min = min(reduced[:,1])
            y_max = max(reduced[:,1])
            
            # update plot bounds
            bx = x_max-x_min
            by = y_max-y_min
            x_min -= (max(bx,by)/bx -1) * bx/2
            x_max += (max(bx,by)/bx -1) * bx/2
            y_min -= (max(bx,by)/by -1) * by/2
            y_max += (max(bx,by)/by -1) * by/2
            bounds_range = max(x_max-x_min, y_max-y_min)
            
            t5 = time.perf_counter()
            # create samples
            def compute_and_predict_grid(dataset, inv_tf_fn, predict_fn, N, x_min, x_max, y_min, y_max):
                px = np.linspace(x_min, x_max, num=N)
                py = np.linspace(y_min, y_max, num=N)
                xx, yy = np.meshgrid(px,py)

                tic = time.perf_counter()
                nn_invp = inv_tf_fn(np.c_[xx.ravel(), yy.ravel()])
                tac = time.perf_counter()

                for i,f in enumerate(dataset.features):
                    if f not in dataset.selected_features:
                        nn_invp = np.insert(nn_invp, i, [point[i]]*len(nn_invp), axis=1)

                probs = predict_fn(nn_invp)
                toc = time.perf_counter()
                #print("inv_points  ",tac-tic,"s")
                #print("predict     ",toc-tac,"s")
                return probs, nn_invp

            for j in range(1,6):    
                nn_probs, nn_invp = compute_and_predict_grid(dataset, nn_pip.inverse_transform, predictor.predict_proba, N, x_min, x_max, y_min, y_max)
                visible_class_count = np.count_nonzero(np.amax(nn_probs, axis=0) > 0.5)
                if visible_class_count >= 2: 
                    break
                print(i, 'needs more iterations:', j)
                x_min -= (j/2.5)*bounds_range
                x_max += (j/2.5)*bounds_range
                y_min -= (j/2.5)*bounds_range
                y_max += (j/2.5)*bounds_range
            
            img = ((np.clip(np.max(nn_probs, axis=1),0.501,0.999) - 0.5) * 2 + np.argmax(nn_probs, axis=1)).reshape((N,N))
            t6 = time.perf_counter()
            # adjust image to local version
            #dataset.datasource.data['x1'] = reduced[:,0]
            #dataset.datasource.data['y1'] = reduced[:,1]
            #sizes = len(dataset.data)*[15]
            #sizes[i] = params.dot_size
            #dataset.datasource.data['size'] = sizes#[params.dot_size if i!=j else 15 for j in range(len(dataset.data))]
            t7 = time.perf_counter()    
            # find next cf-pixel
            pix_pos = ((((nn_pip.transform([point[dataset.selected_features]]) - [x_min,y_min]) / [x_max-x_min, y_max-y_min]) * N) - 0.001).astype(int)[0]
            p_class = point['most_prob_class']
            img_cp = np.logical_not(np.logical_and(p_class < img, img < (p_class + 1)))
            #print('i', i, 'class', p_class, 'p', point, 'img', img[point[1], point[0]], 'img_cp', img_cp[point[1], point[0]])
            cf_pix = bfs(img_cp, pix_pos)
            t8 = time.perf_counter()    
            if cf_pix is None:
                print('could not find cf in image of', i)
                source = p.select('image').data_source
                source.data['image'] = [img.reshape((N,N))]
                source.data['x'] = [x_min]
                source.data['y'] = [y_min]
                source.data['dw'] = [x_max-x_min]
                source.data['dh'] = [y_max-y_min]
                p.x_range.update(start=x_min, end=x_max)
                p.y_range.update(start=y_min, end=y_max)
                cf_pix = pix_pos
            cf_pix_diff[i] = np.linalg.norm(pix_pos - cf_pix) * (1 + (j-1)/2.5)
            cfs[i,:] = nn_invp.reshape(N,N,len(dataset.features))[cf_pix[0],cf_pix[1]]
            cf_diff[i] = np.linalg.norm(nn_pip['scaler'].transform(dataset.data.loc[i,dataset.selected_features].to_numpy().reshape(1,-1))[0] 
                                      - nn_pip['scaler'].transform(nn_invp.reshape(N,N,len(dataset.features))[cf_pix[0],cf_pix[1],important_indices].reshape(1,-1))[0], ord=1)
            if i % 100 == 0:
                print('individual PCA progress:', i,'/', len(dataset.data))
            
            t9 = time.perf_counter()    
            source = p.select('image').data_source
            source.data['image'] = [img.reshape((N,N))]
            source.data['x'] = [x_min]
            source.data['y'] = [y_min]
            source.data['dw'] = [x_max-x_min]
            source.data['dh'] = [y_max-y_min]
            p.x_range.update(start=x_min, end=x_max)
            p.y_range.update(start=y_min, end=y_max)
            if False:
                print('Find NN      ', t2-t1)
                print('Compute Impo.', t3-t2)
                print('Create Emb.  ', t4-t3)
                print('reduce points', t5-t4)
                print('invp+predict ', t6-t5)
                print('pix_pos & img', t7-t6)
                print('find img cf  ', t8-t7)
                print('write diffs  ', t9-t8)
    else: 
        # find next cf-pixel
        img = img_src.reshape((N,N))
        pix_pos = ((((reduced - [x_min,y_min]) / [x_max-x_min, y_max-y_min]) * N) - 0.001).astype(int)
        for i, point in enumerate(pix_pos):
            p_class = dataset.data['most_prob_class'][i]
            img_cp = np.logical_not(np.logical_and(p_class < img, img < (p_class + 1)))
            #print('i', i, 'class', p_class, 'p', point, 'img', img[point[1], point[0]], 'img_cp', img_cp[point[1], point[0]])
            cf_pix = bfs(img_cp, point)
            cf_pix_diff[i] = np.linalg.norm(point - cf_pix)
            cfs[i,:] = invp.reshape(N,N,len(dataset.features))[cf_pix[0],cf_pix[1]]
            cf_diff[i] = np.linalg.norm(predictor['scaler'].transform(dataset.data.loc[i,dataset.features].to_numpy().reshape(1,-1))[0]  - 
                                        predictor['scaler'].transform(invp.reshape(N,N,len(dataset.features))[cf_pix[0],cf_pix[1]].reshape(1,-1))[0], ord=1)
        
    np.save('benchmarks/'+params.dataset_name+'/IMG_cfs_pix_diff_L2_'+params.dataset_name+'_'+params.classifier+'_'+reverse,cf_pix_diff)
    np.save('benchmarks/'+params.dataset_name+'/IMG_cfs_diff_L1_'+params.dataset_name+'_'+params.classifier+'_'+reverse,cf_diff)
    np.save('benchmarks/'+params.dataset_name+'/IMG_cfs_'+params.dataset_name+'_'+params.classifier+'_'+reverse,cfs)

In [ ]:
# find mathematical cf
from alibi.explainers import CounterFactual

shape = (1,) + (len(dataset.features),)
model = predictor.pip['classificator'].model
mins = predictor.pip['scaler'].transform([dataset.data[dataset.features].min(0)])[0]
maxs = predictor.pip['scaler'].transform([dataset.data[dataset.features].max(0)])[0]
ranges = [(i,j) for i,j in zip(mins,maxs)] # (-1e10, 1e10)

# You cannot re-run this cell due to alibi. Their abstract constructor can only be called once.
cf_generator = CounterFactual(model, shape, distance_fn='l1', target_proba=0.56,
                    target_class='other', max_iter=500, early_stop=50, lam_init=0.1,
                    max_lam_steps=20, tol=0.05, learning_rate_init=0.1,
                    feature_range=(-1e10, 1e10), eps=0.05, init='identity',
                    decay=True, write_dir=None, debug=False)
cfs = np.zeros((len(dataset.data),len(dataset.features)))
dists = np.zeros(len(dataset.data))
tic = time.perf_counter()
for index, point in enumerate(predictor.pip['scaler'].transform(dataset.data.loc[:,dataset.features])):
    expl = cf_generator.explain(point.reshape(1,-1))
    if expl.cf is None:
        print(index, 'could not find optimal cf,  img-pos:', predictor.pip['scaler'].inverse_transform(point), ' point', point)
        continue
    else:
        cfs[index] = expl.cf['X']
        dists[index] = expl.cf['distance']
        
    if index % 50 == 0 or index < 10: 
        toc = time.perf_counter()
        print('Found cf for index', index, '/', len(dataset.data),' total time:', int(toc-tic), 's   with L1-distance:', expl.cf['distance'])
        np.save('benchmarks/'+params.dataset_name+'/cfs_L1_'+params.dataset_name+'_'+params.classifier+'.npy', cfs)
        np.save('benchmarks/'+params.dataset_name+'/dists_L1_'+params.dataset_name+'_'+params.classifier+'.npy', dists)
    #predictor.pip['scaler'].inverse_transform(expl.cf['X'])
np.save('benchmarks/'+params.dataset_name+'/cfs_L1_'+params.dataset_name+'_'+params.classifier+'.npy', cfs)
np.save('benchmarks/'+params.dataset_name+'/dists_L1_'+params.dataset_name+'_'+params.classifier+'.npy', dists)

In [ ]:
p = figure(height=params.bot_height, width=params.bot_height+30, tools="pan,tap,box_select,lasso_select", x_range=(-1,1), y_range=(-1,1))
    
p.add_tools(HoverTool(names=["scatter"], name='hover', tooltips = [("id", "$index")]), WheelZoomTool(zoom_on_axis=False))

p.image(image=[[]], color_mapper=LinearColorMapper(palette=params.palette, low=0, high=params.num_colors), 
        name='image', x=[0], y=[0], dw=[0], dh=[0], level='underlay')

update_non_linear_view(p, predictor.pip, method='UMAP', reverse='iLAMP')
update_non_linear_view(p, predictor.pip, method='UMAP', reverse='NNInv')
update_non_linear_view(p, predictor.pip, method='PCA', reverse='PCA')
update_non_linear_view(p, predictor.pip, method='PCA', reverse='PCA_filtered')

mapper = linear_cmap(field_name='sat_color', palette=params.palette, low=0, high=params.num_colors)
p.scatter(source=dataset.datasource, x='x1', y='y1', color='sat_color', line_color='line_color', 
          size='size', name="scatter", nonselection_fill_alpha=0.1, nonselection_line_alpha=0.1)

p.scatter(source=dataset.datasource, x='x1', y='y1', fill_color='target_color',
          line_color='target_color', size='size', name="wrong_scatter",
          nonselection_fill_alpha=0.0, nonselection_line_alpha=0.0, marker='cross',
          view=CDSView(source=dataset.datasource, filters=[dataset.wrong_filter]))

# minimalistic style
p.axis.visible = False
p.xgrid.visible = False
p.ygrid.visible = False
#     p.outline_line_color = None
p.toolbar_location = "right"
p.toolbar.logo = None
p.toolbar.active_scroll = p.select_one(WheelZoomTool)

#pn.pane.Bokeh(p)

In [ ]:
dsets = ['breast', 'diabetes', 'robot24', 'shuttle']
max_d = np.zeros(len(dsets))
avg_L1_diff = np.zeros((len(dsets),4))
parity_rmse = np.zeros((len(dsets),4))
corr = np.zeros((len(dsets),4))
for k, ds in enumerate(dsets):
    print(ds)
    for i, key in enumerate(['iLAMP', 'NNInv', 'PCA', 'PCA_filtered']):
        for network in ['TensorFlow100']:
            x = np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy')
            y = np.load('benchmarks/'+ds+'/IMG_cfs_diff_L1_'+ds+'_'+network+'_'+key+'.npy')
            mask = np.argwhere(x != 0).reshape(-1)
            print("{:.2f}".format(np.mean(y[mask])), 'average L1 diff', key, network, )
            max_d[k] = max(max_d[k], np.amax(y[mask]))
            avg_L1_diff[k,i] = np.mean(y[mask])
            parity_rmse[k,i] = mean_squared_error(x, y, squared=False)
            corr[k,i] = np.corrcoef(np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy')[mask], np.load('benchmarks/'+ds+'/IMG_cfs_diff_L1_'+ds+'_'+network+'_'+key+'.npy')[mask])[0,1]
            #print('avg pix', key, network, 'L2 diff', np.mean(np.load('benchmarks/'+ds+'/IMG_cfs_pix_diff_L2_'+ds+'_'+network+'_'+key+'.npy')[mask]))
            #print('average', key, 'L1 diff', np.load('IMG_cfs_diff_L1_'+ds+'_TensorFlow100_'+key+'.npy')[:10])
            #print(key, 'cfs', np.load('IMG_cfs_diabetes_TensorFlow100_'+key+'.npy')[index])
            #print(key, 'cfs_back', predictor.pip['scaler'].inverse_transform(np.load('IMG_cfs_diabetes_TensorFlow100_'+key+'.npy')[:1]))
    math_cfs = np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy')
    print("{:.2f}".format(np.mean(math_cfs[math_cfs != 0])), 'average L1 diff alibi')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi']= 150
plt.style.use('seaborn-whitegrid')
from scipy.stats import linregress
from scipy.optimize import curve_fit

fig, axes = plt.subplots(len(dsets), 4, figsize=(10,11), sharey='row')
fig.suptitle('Parity plots of L1-distances to nearest counterfactual')
x_dat = 'dists_L1' # dists_L1, IMG_cfs_pix_diff_L2, IMG_cfs_diff_L1
y_dat = 'IMG_cfs_diff_L1'
for k, ds in enumerate(dsets):
    for i, key in enumerate(['iLAMP', 'NNInv', 'PCA', 'PCA_filtered']):
        for j, network in enumerate(['TensorFlow100']):
            if k == 0:
                axes[k][i].set_title(key)
                if key == 'PCA':
                    axes[k][i].set_title('CoFFi')
                if key == 'PCA_filtered':
                    axes[k][i].set_title('filtered CoFFi')
                    
            mask = np.argwhere(np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy') != 0).reshape(-1)
            if x_dat == 'dists_L1':
                x = np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy')[mask]
                if k == len(dsets)-1:
                    axes[k][i].set_xlabel('$d_{optimization}$')#'L1-distance to nearest CF')
            elif x_dat == 'IMG_cfs_pix_diff_L2':
                x = np.load('benchmarks/'+ds+'/'+x_dat+'_'+ds+'_'+network+'_'+key+'.npy')[mask]
                if k == len(dsets)-1:
                    axes[k][i].set_xlabel('pixel-distance to nearest shown CF')
            elif x_dat == 'IMG_cfs_diff_L1':
                x = np.load('benchmarks/'+ds+'/'+x_dat+'_'+ds+'_'+network+'_'+key+'.npy')[mask]
                if k == len(dsets)-1:
                    axes[k][i].set_xlabel('L1-distance to nearest shown CF')
                
            if y_dat == 'dists_L1':
                y = np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy')[mask]
                axes[k][0].set_ylabel('L1-distance to nearest CF')
            elif y_dat == 'IMG_cfs_pix_diff_L2':
                y = np.load('benchmarks/'+ds+'/'+y_dat+'_'+ds+'_'+network+'_'+key+'.npy')[mask]
                axes[k][0].set_ylabel('pixel-distance to nearest shown CF')
            elif y_dat == 'IMG_cfs_diff_L1':
                y = np.load('benchmarks/'+ds+'/'+y_dat+'_'+ds+'_'+network+'_'+key+'.npy')[mask]
                axes[k][0].set_ylabel('$d_{shown}$')#'L1-distance to nearest shown CF')
            
            # write parity RMSE compared to cf found through optimization (with alibi)
            rmse_str = "{:.2f}".format(parity_rmse[k,i])
            if np.amin(parity_rmse[k,:3]) == parity_rmse[k,i]:
                rmse_str = "\mathbf{"+rmse_str+"}"
            axes[k][i].text(np.max(x),max_d[k]*0.85,"$RMSE="+rmse_str+"$", horizontalalignment='right')
            
            # write correlation between shown and alibi cf
            corr_str = "{:.2f}".format(corr[k,i])
            if np.amax(corr[k,:3]) == corr[k,i]:
                corr_str = "\mathbf{"+corr_str+"}"
            axes[k][i].text(np.max(x),max_d[k]*0.75,r"$\rho="+corr_str+"$", horizontalalignment='right')
            
            # write average distance of shown cf
            avg_str = "{:.2f}".format(avg_L1_diff[k,i])
            if np.amin(avg_L1_diff[k,:3]) == avg_L1_diff[k,i]:
                avg_str = "\mathbf{"+avg_str+"}"
            axes[k][i].text(np.max(x),max_d[k]*0.95,r"$\overline{d_{shown}}="+avg_str+"$", horizontalalignment='right')
            
            x1=np.linspace(0,np.max(x),500)
            
            gradient, intercept, r_value, p_value, std_err = linregress(x,y)
            y3=gradient*x1+intercept
            #print('regress error', ((gradient*x+intercept - y)**2).mean())
            
            gradient, residuals, r_value, _ = np.linalg.lstsq(x.reshape(-1,1),y,rcond=None)
            y1=gradient*x1
            #print('linear error ', ((gradient*x - y)**2).mean()) ### THIS IS GOOD
            
            popt, pcov = curve_fit(lambda x,a: a*np.sqrt(x), x, y, p0=1)
            y2=popt*np.sqrt(x1)
            #popt, pcov = curve_fit(lambda x,a: a*x**2, x, y, p0=1)
            #y1=popt*x1**2
            #print('non-lin error', ((popt*np.sqrt(x) - y)**2).mean())
            
            # Define a function (quadratic in our case) to fit the data with.
            #def f(B, x):
            #    return B[0]*x + B[1]
            # Create a RealData object using our initiated data from above.
            #data = RealData(x, y)
            # Set up ODR with the model and data.
            #odr = ODR(data, Model(f), beta0=[0, 0])
            # Run the regression.
            #out = odr.run()
            #y1=out.beta[0]*x1+out.beta[1]
            #print(list(out.beta))

            #axes[i].plot(x, y, 'o', color='black', alpha=0.2, markersize=2)
            axes[k][i].plot(x1,x1,'-k', alpha=0.5)
            axes[k][i].plot(x, y, 'o', color='black', alpha=0.2, markersize=3, markeredgewidth=0)
            #axes[k][i].plot(x1,y1,'-r')
            #axes[i].plot(x1,y2,'-g')
            #axes[i].plot(x1,y3,'-b')
# Set common labels
#fig.text(0.5, 0.04, 'common xlabel', ha='center', va='center')
    fig.text(0.01, 1 - (k / (len(dsets)+0.35) + 0.63 / (len(dsets))), ds, ha='center', va='center', rotation='vertical')
plt.tight_layout()
plt.savefig("./benchmarks.png",dpi=300)

In [ ]:
#math_dists = np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy')
#print(np.sort(math_dists)[:50])
#print(dataset.data.loc[np.where(math_dists < 0.2), 'prob'])

for k, ds in enumerate(dsets):
    print(ds)
    for j, network in enumerate(['TensorFlow100']):
        for i, key in enumerate(['NNInv', 'iLAMP', 'PCA', 'PCA_filtered']):
            mask = np.argwhere(np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy') != 0).reshape(-1)
            #print('corr  math-CF to pix-diff', network, key, np.corrcoef(np.load('benchmarks/'+params.dataset_name+'/dists_L1_'+ds+'_'+network+'.npy'), np.load('benchmarks/'+params.dataset_name+'/IMG_cfs_pix_diff_L2_'+ds+'_'+network+'_'+key+'.npy'))[0,1])   

            #print('corr image-CF to pix-diff', network, key, np.corrcoef(np.load('benchmarks/'+ds+'/IMG_cfs_diff_L1_'+ds+'_'+network+'_'+key+'.npy'), np.load('benchmarks/'+ds+'/IMG_cfs_pix_diff_L2_'+ds+'_'+network+'_'+key+'.npy'))[0,1])
            
            #cov = np.cov(np.array([np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy'), np.load('benchmarks/'+ds+'/IMG_cfs_diff_L1_'+ds+'_'+network+'_'+key+'.npy')]))
            #print(cov[0,1] / (np.sqrt(cov[0,0])*np.sqrt(cov[1,1])))
            
            print('corr  math-CF to image-CF', network, key, np.corrcoef(np.load('benchmarks/'+ds+'/dists_L1_'+ds+'_TensorFlow100.npy')[mask], np.load('benchmarks/'+ds+'/IMG_cfs_diff_L1_'+ds+'_'+network+'_'+key+'.npy')[mask])[0,1])